# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-84291ccba1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Monarchies") 

Consider the following exploratory information need:

> You investigating existing and past historical monarchies

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P1366`   | replaced by    | predicate |
| `wdt:P122`    | basic form of government    | predicate |
| `wd:Q38`      | Italy          | node |
| `wd:Q145`     | United Kingdom           | node |
| `wd:Q7269`    | monarchy           | node |




Also consider

```
?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
```

is the BGP to retrieve all **countries that are some instance of a monarchy**

## Workload Goals


1. Identify the BGP to retrieve states that were monarchies and have been replaced by some other state

2. Identify the BGP for obtaining statistical information of monarchies, e.g., poluation or inception date

3. Which is or was  the country with the highest population?

4. How many countries are or have been monarchies across continents?

5. Analyze the number of countries that were monarchies
 
   5.1 How many sub-types of monarchy exist?  How many instances of exist of each one?
   
   5.2 Are there cities that have been capital of multiple kingdoms?
   
   5.3 Which are the top-3 monarchies with the earliest inception and still existing?


In [1]:
# start your workflow here

In [7]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2344')]


1

## Task 1 - Identify the BGP to retrieve states that were monarchies and have been replaced by some other state

#### I inspect Properties of the Monarchies

In [5]:
queryString = """
SELECT DISTINCT ?p ?pname (COUNT(?p) AS ?number) WHERE { 

?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
     ?p ?obj.
    
?p <http://schema.org/name> ?pname.
} 
GROUP BY ?p ?pname
ORDER BY DESC (?number)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pname', 'language used'), ('number', '4945')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('number', '3785')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pname', 'diplomatic relation'), ('number', '3052')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of'), ('number', '2141')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administrative territorial entity'), ('number', '1711')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('pname', 'capital'), ('number', '1511')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pname', 'inception'), ('number', '1474')]
[('p', 'http://www.wikidata.org/prop/direct/P576'), ('pname', 'dissolved, abolished or demolished date'), ('number', '1332')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID'), ('number', '1290')]
[('p', 'http://www.wikidata.org/prop/direct/P122')

100

#### I use "replaced by" (wdt:P1366)

In [ ]:
queryString = """
SELECT DISTINCT ?name ?Replace WHERE { 

?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P1366 ?m.
    
?mon <http://schema.org/name> ?name.
?m <http://schema.org/name> ?Replace.

} 
"""

print("Results")
run_query(queryString)

## Task 2 - Identify the BGP for obtaining statistical information of monarchies, e.g., population or inception date


#### I use "inception"(wdt:P571), "currency" (wdt:P38), "population" (wdt:P1082), "area" (wdt:P2046)

In [12]:
queryString = """
SELECT DISTINCT ?name ?inc ?curname (MAX(?pop) AS ?latest_pop) ?area WHERE { 

?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269.

OPTIONAL{?mon wdt:P571 ?inc.}
OPTIONAL{?mon wdt:P38 ?cur.}
OPTIONAL{?mon wdt:P1082 ?pop.}
OPTIONAL{?mon wdt:P2046 ?area.}
    
?mon <http://schema.org/name> ?name.
?cur <http://schema.org/name> ?curname.
} 
GROUP BY ?name ?inc ?curname ?area
"""

print("Results")
run_query(queryString)

Results
[('name', 'Grand Duchy of Lithuania'), ('inc', '1236-01-01T00:00:00Z'), ('curname', 'groschen'), ('latest_pop', '3500000'), ('area', '930000')]
[('name', 'Kingdom of Kongo'), ('inc', '1395-01-01T00:00:00Z'), ('curname', 'Shell money'), ('area', '129400')]
[('name', 'Grand Duchy of Lithuania'), ('inc', '1236-01-01T00:00:00Z'), ('curname', 'złoty'), ('latest_pop', '3500000'), ('area', '250000')]
[('name', 'Thailand'), ('inc', '1768-12-28T00:00:00Z'), ('curname', 'baht'), ('latest_pop', '65931550'), ('area', '513119.5')]
[('name', 'Greenland'), ('inc', '1979-01-01T00:00:00Z'), ('curname', 'Danish krone'), ('latest_pop', '56081'), ('area', '2166086')]
[('name', 'Grand Duchy of Lithuania'), ('inc', '1236-01-01T00:00:00Z'), ('curname', 'Lithuanian long currency'), ('latest_pop', '3500000'), ('area', '320000')]
[('name', 'Belize'), ('inc', '1981-09-21T00:00:00Z'), ('curname', 'Belize dollar'), ('latest_pop', '374681'), ('area', '22966')]
[('name', 'Principality of Sealand'), ('inc', '

425

#### I see that there are a lot of repetitions as some monarchies have more inception dates, so i clean the results to get the oldest inception date and latest measurement of the population

In [24]:
queryString = """
SELECT DISTINCT ?name ?date ?latest_pop (GROUP_CONCAT(?curname;separator=",") as ?currencies)  WHERE { 

?mon wdt:P38 ?cur.
#OPTIONAL{?mon wdt:P2046 ?area.}

#Subquery to find the oldest inception date
{SELECT ?mon (MIN(?inc) AS ?date) WHERE {
        ?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
             wdt:P571 ?inc.
}GROUP BY ?mon
}

#Subquery to find the latest population
{SELECT ?mon2 (MAX(?pop) AS ?latest_pop) WHERE {
        ?mon2 (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
               wdt:P1082 ?pop.
}GROUP BY ?mon2
}

?mon <http://schema.org/name> ?name.
?cur <http://schema.org/name> ?curname.
FILTER(?mon=?mon2)
} 

"""

print("Results")
run_query(queryString)

Results
[('name', 'Bahrain'), ('date', '1971-08-14T00:00:00Z'), ('latest_pop', '1492584'), ('currencies', 'Bahraini dinar')]
[('name', 'United Kingdom of Great Britain and Ireland'), ('date', '1801-01-01T00:00:00Z'), ('latest_pop', '45370530'), ('currencies', 'pound sterling')]
[('name', 'Kingdom of Italy'), ('date', '1861-03-17T00:00:00Z'), ('latest_pop', '42399000'), ('currencies', 'Italian lira')]
[('name', 'Saint Vincent and the Grenadines'), ('date', '1979-01-01T00:00:00Z'), ('latest_pop', '109897'), ('currencies', 'Eastern Caribbean dollar')]
[('name', 'Malaysia'), ('date', '1957-01-01T00:00:00Z'), ('latest_pop', '31624264'), ('currencies', 'Malaysian ringgit')]
[('name', 'Saint Kitts and Nevis'), ('date', '1967-02-27T00:00:00Z'), ('latest_pop', '55345'), ('currencies', 'Eastern Caribbean dollar')]
[('name', 'Byzantine Empire'), ('date', '0395-01-18T00:00:00Z'), ('latest_pop', '17000000'), ('currencies', 'Byzantine coinage,ducat,solidus')]
[('name', 'Bogd Khanate'), ('date', '191

124

#### I find for each monarchy inception(wdt:P571), population(wdt:P1082) and area(wdt:P2046)

In [29]:
queryString = """
SELECT DISTINCT ?name ?inception ?latest_pop ?area WHERE { 

#Subquery to find the oldest inception date
{SELECT ?mon (MIN(?inc) AS ?inception) WHERE {
        ?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
             wdt:P571 ?inc.
}GROUP BY ?mon
}

#Subquery to find the latest population
{SELECT ?mon2 (MAX(?pop) AS ?latest_pop) WHERE {
        ?mon2 (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
               wdt:P1082 ?pop.
}GROUP BY ?mon2
}

#Subquery to find the maximum area
{SELECT ?mon3 (MAX(?a) AS ?area) WHERE {
        ?mon3 (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
               wdt:P2046 ?a.
}GROUP BY ?mon3
}

?mon <http://schema.org/name> ?name.
FILTER(?mon=?mon2 && ?mon2=?mon3)
} 

"""

print("Results")
run_query(queryString)

Results
[('name', 'Kingdom of the Netherlands'), ('inception', '1815-03-16T00:00:00Z'), ('latest_pop', '17100715'), ('area', '42201')]
[('name', 'Austro-Hungarian rule in Bosnia and Herzegovina'), ('inception', '1878-01-01T00:00:00Z'), ('latest_pop', '1898044'), ('area', '51082')]
[('name', 'Kingdom of Libya'), ('inception', '1951-12-24T00:00:00Z'), ('latest_pop', '1091830'), ('area', '1759530')]
[('name', 'Principality of Sealand'), ('inception', '1967-09-02T00:00:00Z'), ('latest_pop', '27'), ('area', '0.004')]
[('name', 'Saskatchewan'), ('inception', '1905-01-01T00:00:00Z'), ('latest_pop', '1168057'), ('area', '651900')]
[('name', 'Emirate of Ras Al Khaimah'), ('inception', '1869-01-01T00:00:00Z'), ('latest_pop', '241000'), ('area', '1684')]
[('name', 'Austria-Hungary'), ('inception', '1867-03-01T00:00:00Z'), ('latest_pop', '52800000'), ('area', '676615')]
[('name', 'First Toungoo Empire'), ('inception', '1510-10-26T00:00:00Z'), ('latest_pop', '6000000'), ('area', '1550000')]
[('name

169

## Task 3 -Which is or was the country with the highest population?

#### I reuse one query found before

In [31]:
queryString= """
#Subquery to find the latest population
SELECT ?country ?pop WHERE {

?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
      wdt:P1082 ?pop.

?mon <http://schema.org/name> ?country.
}
ORDER BY DESC (?pop)
LIMIT 1

"""

print("Results")
run_query(queryString)

Results
[('country', 'British Empire'), ('pop', '680000000')]


1

#### Plausible as the British Empire ruled from Canada to Australia

## Task 4 - How many countries are or have been monarchies across continents?

#### From the initial search i notice the property "continent" (wdt:P30), let's see some values

In [ ]:
queryString= """
SELECT ?country ?continent WHERE {

?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
      wdt:P30 ?con.

?mon <http://schema.org/name> ?country.
?con <http://schema.org/name> ?continent.
}
"""

print("Results")
run_query(queryString)

#### Perfect, i count them grouping by continent

In [5]:
queryString= """
SELECT ?continent (COUNT(DISTINCT ?mon) AS ?number) WHERE {

?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
      wdt:P30 ?con.

?mon <http://schema.org/name> ?country.
?con <http://schema.org/name> ?continent.

}
GROUP BY ?continent
ORDER BY DESC (?number)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'Europe'), ('number', '361')]
[('continent', 'Asia'), ('number', '173')]
[('continent', 'Africa'), ('number', '63')]
[('continent', 'North America'), ('number', '31')]
[('continent', 'Eurasia'), ('number', '14')]
[('continent', 'Insular Oceania'), ('number', '13')]
[('continent', 'Hyboria'), ('number', '13')]
[('continent', 'South America'), ('number', '8')]
[('continent', 'The Continent'), ('number', '7')]
[('continent', 'Thuria'), ('number', '6')]
[('continent', 'Americas'), ('number', '5')]
[('continent', 'Central America'), ('number', '2')]
[('continent', 'Australia'), ('number', '2')]
[('continent', 'Antarctica'), ('number', '1')]
[('continent', 'Northern America and Mexico'), ('number', '1')]
[('continent', 'Caribbean'), ('number', '1')]


16

#### Now i want to try to count how many ACTUAL countries are or have been monarchies

#### I start by listing  the entities that are instance of "country"(wdt:P17) to check if the entities are only countries that exists nowadays

In [9]:
###WRONG QUERY, COUNTRY IS A PREDICATE NOT AN ENTITY
queryString= """
SELECT DISTINCT ?countryname WHERE {

?country wdt:P31 wd:P17.

?country <http://schema.org/name> ?countryname.

}

"""

print("Results")
run_query(queryString)

Results
Empty


0

#### Let's try to get all "instance of" of Italy to search for the country class

In [10]:

queryString= """
SELECT DISTINCT ?obj ?objname WHERE {

wd:Q38 wdt:P31 ?obj.

?obj <http://schema.org/name> ?objname.
}

"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3624078'), ('objname', 'sovereign state')]
[('obj', 'http://www.wikidata.org/entity/Q6256'), ('objname', 'country')]


2

#### Let's print all entities that are instance of the found "country"(wd:Q6256)

In [12]:
queryString= """
SELECT DISTINCT ?countryname WHERE {

?country wdt:P31 wd:Q6256.

?country <http://schema.org/name> ?countryname.
}

"""

print("Results")
run_query(queryString)

Results
[('countryname', 'Northern Mariana Islands')]
[('countryname', 'Pahlavi Dynasty')]
[('countryname', 'Danish Realm')]
[('countryname', 'Dominican Republic')]
[('countryname', 'Transnistria')]
[('countryname', 'Republic of Artsakh')]
[('countryname', 'The Gambia')]
[('countryname', 'Guinea')]
[('countryname', 'Guinea-Bissau')]
[('countryname', 'Ivory Coast')]
[('countryname', 'Cameroon')]
[('countryname', 'Cape Verde')]
[('countryname', 'Lesotho')]
[('countryname', 'Liberia')]
[('countryname', 'Libya')]
[('countryname', 'Madagascar')]
[('countryname', 'Malawi')]
[('countryname', 'Mauritania')]
[('countryname', 'Mauritius')]
[('countryname', 'Morocco')]
[('countryname', 'Mozambique')]
[('countryname', 'Namibia')]
[('countryname', 'Niger')]
[('countryname', 'Nigeria')]
[('countryname', 'Uganda')]
[('countryname', 'Rwanda')]
[('countryname', 'São Tomé and Príncipe')]
[('countryname', 'Senegal')]
[('countryname', 'Seychelles')]
[('countryname', 'Sierra Leone')]
[('countryname', 'Soma

190

#### I inspect their properties to try to remove the little noise in these data (countries that doesn't exists anymore)

In [ ]:
queryString= """
SELECT DISTINCT ?p ?pname (COUNT(?p) AS ?number) WHERE {

?country wdt:P31 wd:Q6256;
         ?p ?obj.

?p <http://schema.org/name> ?pname.
}
GROUP BY ?p ?pname
ORDER BY DESC(?number)
"""

print("Results")
run_query(queryString)

#### I found "dissolved, abolished or demolished state"(wd:P576),so i exclude them from the list

In [ ]:
queryString= """
SELECT DISTINCT ?countryname WHERE {

?country wdt:P31 wd:Q6256.

?country <http://schema.org/name> ?countryname.
FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}

"""

print("Results")
run_query(queryString)

#### Subquery, i try to retrieve current country that are monarchies right now (so they don't have the "dissolved..." property)

In [19]:
queryString= """
SELECT DISTINCT ?countryname WHERE {

?country wdt:P31 wd:Q6256;
         (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269.

?country <http://schema.org/name> ?countryname.
FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}

"""

print("Results")
run_query(queryString)

Results
[('countryname', 'Danish Realm')]
[('countryname', 'Lesotho')]
[('countryname', 'Morocco')]
[('countryname', 'United Kingdom')]
[('countryname', 'Canada')]
[('countryname', 'Japan')]
[('countryname', 'Norway')]
[('countryname', 'Andorra')]
[('countryname', 'Kingdom of the Netherlands')]
[('countryname', 'Sweden')]
[('countryname', 'Liechtenstein')]
[('countryname', 'Denmark')]
[('countryname', 'Bahrain')]
[('countryname', 'Australia')]
[('countryname', 'Cambodia')]
[('countryname', 'Netherlands')]
[('countryname', 'New Zealand')]
[('countryname', 'Papua New Guinea')]
[('countryname', 'Saint Vincent and the Grenadines')]
[('countryname', 'Saint Lucia')]
[('countryname', 'Saint Kitts and Nevis')]
[('countryname', 'The Bahamas')]
[('countryname', 'Jordan')]
[('countryname', 'Malaysia')]
[('countryname', 'Oman')]
[('countryname', 'Qatar')]
[('countryname', 'Saudi Arabia')]
[('countryname', 'Thailand')]
[('countryname', 'United Arab Emirates')]
[('countryname', 'Bhutan')]
[('country

38

#### Now i try to get countries that now are not monarchies but they were in the past, by using "replaced by"(wdt:P1366)

In [24]:
queryString= """
SELECT DISTINCT ?countryname WHERE {

?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
      wdt:P1366+ ?country.     

?country wdt:P31 wd:Q6256.

?country <http://schema.org/name> ?countryname.
FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}

"""

print("Results")
run_query(queryString)

Results
[('countryname', "People's Republic of China")]
[('countryname', 'Japan')]
[('countryname', 'Croatia')]
[('countryname', 'Andorra')]
[('countryname', 'Montenegro')]
[('countryname', 'Hungary')]
[('countryname', 'Liechtenstein')]
[('countryname', 'Italy')]
[('countryname', 'Turkey')]
[('countryname', 'Taiwan')]
[('countryname', 'India')]
[('countryname', 'Haiti')]
[('countryname', 'Iran')]
[('countryname', 'United Arab Emirates')]
[('countryname', 'Portugal')]
[('countryname', 'Uzbekistan')]
[('countryname', 'Tajikistan')]
[('countryname', 'Turkmenistan')]
[('countryname', 'United Kingdom')]
[('countryname', 'Brazil')]
[('countryname', 'Ireland')]
[('countryname', 'Pakistan')]
[('countryname', 'Tanzania')]
[('countryname', 'Libya')]
[('countryname', 'Albania')]
[('countryname', 'Malta')]
[('countryname', 'Indonesia')]
[('countryname', 'South Africa')]
[('countryname', 'Finland')]
[('countryname', 'Denmark')]
[('countryname', 'Bahrain')]
[('countryname', 'Netherlands')]
[('countr

60

#### Now i fuse the two subqueries and aggregate the country by continent

In [28]:
queryString= """
SELECT DISTINCT ?continent (COUNT(?country) AS ?number) WHERE {

?country wdt:P30 ?con.

#Subquery for finding the countries
{
SELECT DISTINCT ?country WHERE {
{
?mon (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
      wdt:P1366+ ?country.     
?country wdt:P31 wd:Q6256.
FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}
UNION
{
?country wdt:P31 wd:Q6256;
         (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269.
FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}
}
}
?con <http://schema.org/name> ?continent.
}
GROUP BY ?continent
ORDER BY DESC (?number)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'Asia'), ('number', '30')]
[('continent', 'Europe'), ('number', '20')]
[('continent', 'Africa'), ('number', '18')]
[('continent', 'North America'), ('number', '13')]
[('continent', 'Insular Oceania'), ('number', '7')]
[('continent', 'South America'), ('number', '3')]
[('continent', 'Central America'), ('number', '2')]
[('continent', 'Eurasia'), ('number', '1')]


8

## Task 5.1 How many sub-types of monarchy exist? How many instances of exist of each one?

#### Let's try to get all subclasses of the monarchy government type

In [6]:
queryString = """
SELECT DISTINCT ?subname WHERE { 

?p (wdt:P31|wdt:P122) ?o.
 ?o wdt:P279* wd:Q7269. 

?o <http://schema.org/name> ?subname.
} 
"""

print("Results")
run_query(queryString)

Results
[('subname', 'patrimonialism')]
[('subname', 'feudal monarchy')]
[('subname', 'despotism')]
[('subname', 'elective monarchy')]
[('subname', 'hereditary monarchy')]
[('subname', 'absolute monarchy')]
[('subname', 'federal monarchy')]
[('subname', 'parliamentary monarchy')]
[('subname', 'constitutional monarchy')]
[('subname', 'monarchy')]
[('subname', 'kingdom')]
[('subname', 'polysynodal regime')]
[('subname', 'principality')]
[('subname', 'Bourbon Reforms')]
[('subname', 'regency')]
[('subname', 'parliamentary coprincipality')]
[('subname', 'Tanistry')]
[('subname', 'composite monarchy')]
[('subname', 'abbey-principality')]
[('subname', 'Electoral Principate')]
[('subname', 'ecclesiastical principality')]
[('subname', 'crown land of the Austrian Empire')]
[('subname', 'Hochstift')]
[('subname', 'duchy')]
[('subname', 'petty kingdom')]
[('subname', 'list of Indonesian monarchies')]
[('subname', 'part principality')]
[('subname', 'grand duchy')]
[('subname', 'Duchies of Silesia'

44

#### I try to get only the last step, to see if results are better

In [7]:
queryString = """
SELECT DISTINCT ?subname WHERE { 

 ?o wdt:P279 wd:Q7269. 

?o <http://schema.org/name> ?subname.
} 
"""

print("Results")
run_query(queryString)

Results
[('subname', 'patrimonialism')]
[('subname', 'feudal monarchy')]
[('subname', 'despotism')]
[('subname', 'elective monarchy')]
[('subname', 'hereditary monarchy')]
[('subname', 'absolute monarchy')]
[('subname', 'federal monarchy')]
[('subname', 'constitutional monarchy')]
[('subname', 'duchy')]
[('subname', 'kingdom')]
[('subname', 'list of Indonesian monarchies')]
[('subname', 'monarchy of Barbados')]
[('subname', 'principality')]
[('subname', 'empire')]
[('subname', 'self-proclaimed monarchy')]
[('subname', 'regency')]
[('subname', 'emperor system')]
[('subname', 'tsarism')]
[('subname', 'Monarchy of Liechtenstein')]
[('subname', 'divine rights monarchy')]
[('subname', 'monarchies in Europe')]
[('subname', 'Tanistry')]
[('subname', 'composite monarchy')]
[('subname', 'Semi-constitutional monarchy')]
[('subname', 'Monarchies in Africa')]
[('subname', 'Pre-modern Korea')]


26

#### A better subset, let's count them...I reuse a query for last task to get countries that are monarchies right now and let's count by their type

In [4]:
queryString= """
SELECT DISTINCT ?type (COUNT(?country) AS ?number) WHERE {

?country wdt:P31 wd:Q6256;
         (wdt:P31|wdt:P122)/wdt:P279* ?o.

?o wdt:P279 wd:Q7269. 
?o <http://schema.org/name> ?type.

FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}
GROUP BY(?type)
ORDER BY DESC (?number)
"""

print("Results")
run_query(queryString)

Results
[('type', 'constitutional monarchy'), ('number', '34')]
[('type', 'kingdom'), ('number', '19')]
[('type', 'absolute monarchy'), ('number', '4')]
[('type', 'federal monarchy'), ('number', '3')]
[('type', 'elective monarchy'), ('number', '1')]


5

#### We notice that right now the majority of the monarchies are constitutionals one.

### Task - 5.2 Are there cities that have been capital of multiple kingdoms?

#### I use the property "capital" (wdt:P36) found at the beginning

In [26]:
queryString= """
SELECT DISTINCT ?capitalname (COUNT (DISTINCT ?p) AS ?kingdoms) WHERE{

?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
   wdt:P36 ?capital.

?capital <http://schema.org/name> ?capitalname.  
}GROUP BY (?capitalname)
HAVING ((COUNT (DISTINCT ?p)) > 1)
ORDER BY DESC (?kingdoms)

"""
print("Results")
run_query(queryString)

Results
[('capitalname', 'London'), ('kingdoms', '8')]
[('capitalname', 'Kutaisi'), ('kingdoms', '7')]
[('capitalname', 'Madrid'), ('kingdoms', '7')]
[('capitalname', 'Rio de Janeiro'), ('kingdoms', '7')]
[('capitalname', 'Vienna'), ('kingdoms', '6')]
[('capitalname', 'Paris'), ('kingdoms', '6')]
[('capitalname', 'Cairo'), ('kingdoms', '6')]
[('capitalname', 'Prague'), ('kingdoms', '5')]
[('capitalname', 'Belgrade'), ('kingdoms', '5')]
[('capitalname', 'Riyadh'), ('kingdoms', '5')]
[('capitalname', 'Berlin'), ('kingdoms', '5')]
[('capitalname', 'Bago'), ('kingdoms', '4')]
[('capitalname', 'Naples'), ('kingdoms', '4')]
[('capitalname', 'Vilnius'), ('kingdoms', '4')]
[('capitalname', 'Palermo'), ('kingdoms', '4')]
[('capitalname', 'Kraków'), ('kingdoms', '4')]
[('capitalname', 'Turin'), ('kingdoms', '4')]
[('capitalname', 'Rome'), ('kingdoms', '4')]
[('capitalname', 'Florence'), ('kingdoms', '4')]
[('capitalname', 'Tbilisi'), ('kingdoms', '4')]
[('capitalname', 'Lahore'), ('kingdoms', '4

171

### Task - 5.3 Which are the top-3 monarchies with the earliest inception and still existing?

#### I reuse a country made before to obtain the countries that are monarchies right now and i pick the oldest one by using the property found at the beginning "inception" (wdt:P571)

In [27]:
queryString= """
SELECT DISTINCT ?countryname ?inception WHERE {

?country wdt:P31 wd:Q6256;
         (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
         wdt:P571 ?inception.

?country <http://schema.org/name> ?countryname.
FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}

"""

print("Results")
run_query(queryString)

Results
[('countryname', 'Danish Realm'), ('inception', '0800-01-01T00:00:00Z')]
[('countryname', 'Lesotho'), ('inception', '1966-01-01T00:00:00Z')]
[('countryname', 'Morocco'), ('inception', '-399-01-01T00:00:00Z')]
[('countryname', 'United Kingdom'), ('inception', '1927-04-12T00:00:00Z')]
[('countryname', 'Canada'), ('inception', '1867-07-01T00:00:00Z')]
[('countryname', 'Japan'), ('inception', '1947-05-03T00:00:00Z')]
[('countryname', 'Norway'), ('inception', '0900-01-01T00:00:00Z')]
[('countryname', 'Norway'), ('inception', '1814-05-17T00:00:00Z')]
[('countryname', 'Norway'), ('inception', '1905-06-07T00:00:00Z')]
[('countryname', 'Norway'), ('inception', '1905-10-26T00:00:00Z')]
[('countryname', 'Andorra'), ('inception', '1278-01-01T00:00:00Z')]
[('countryname', 'Kingdom of the Netherlands'), ('inception', '1815-03-16T00:00:00Z')]
[('countryname', 'Sweden'), ('inception', '0700-01-01T00:00:00Z')]
[('countryname', 'Liechtenstein'), ('inception', '1806-07-12T00:00:00Z')]
[('countryn

44

In [29]:
queryString= """
SELECT DISTINCT ?country ?countryname  WHERE {

?country wdt:P31 wd:Q6256;
         (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
         wdt:P571 ?inception.

?country <http://schema.org/name> ?countryname.
FILTER NOT EXISTS{?country wdt:P576 ?obj.}
}
ORDER BY ASC (?inception)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryname', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q756617'), ('countryname', 'Danish Realm')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryname', 'Denmark')]


3

#### I remove one instance of Denmark to have 3 different countries

In [30]:
queryString= """
SELECT DISTINCT ?country ?countryname  WHERE {

?country wdt:P31 wd:Q6256;
         (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
         wdt:P571 ?inception.

?country <http://schema.org/name> ?countryname.
FILTER (NOT EXISTS{?country wdt:P576 ?obj.} && ?country!=wd:Q756617)
}
ORDER BY ASC (?inception)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryname', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryname', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('countryname', 'Norway')]


3

#### Se we see that the oldest monarchies reported are 3 countries from northern Europe